In [1]:
import os
import numpy as np
import pandas as pd
import requests
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import sys
sys.path.append('/home/adedapo/personal_project/daps05ayoade/disseration')
from algotrading.ml_logic.secrecy import API_TOKEN

In [2]:
data_dir = "/home/adedapo/personal_project/daps05ayoade/disseration/data"
file_name = "50_tech_stocks.csv"
file_path = os.path.join(data_dir, file_name)
symbols_df = pd.read_csv(file_path)
tickers = [ticker for ticker in symbols_df['Ticker']]

In [3]:
import requests
import pandas as pd
from algotrading.strategy.data_fetcher import DataFetcher
from algotrading.ml_logic.secrecy import API_TOKEN

data_fetcher = DataFetcher(API_TOKEN)

def get_sentiment_data(symbol, API_TOKEN):
    url = f'https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers={symbol}&apikey={API_TOKEN}'
    response = requests.get(url)
    response.raise_for_status()
    data = data_fetcher.get_data_from_url(url) # Utilize rate limiting from DataFetcher
    ticker_data_list = []

    for feed_item in data['feed']:
        for ticker_data in feed_item["ticker_sentiment"]:
            if ticker_data['ticker'] == symbol:
                ticker_item = {
                    'ticker': symbol,
                    'relevance_score': float(ticker_data['relevance_score']),
                    'ticker_sentiment_score': float(ticker_data['ticker_sentiment_score']),
                    'ticker_sentiment_label': ticker_data['ticker_sentiment_label'],
                }
                ticker_data_list.append(ticker_item)

    sentiment_df = pd.DataFrame(ticker_data_list)

    sentiment_mapping = {
        "Very-Bullish": 5,
        "Bullish": 4,
        "Somewhat-Bullish": 3,
        "Neutral": 2,
        "Somewhat-Bearish": 1,
        "Bearish": 0,
        "Very-Bearish": -1
    }

    sentiment_df['SentimentScore'] = sentiment_df['ticker_sentiment_label'].map(sentiment_mapping)
    sentiment_df['SentimentScore'] = sentiment_df['SentimentScore'].astype(int)
    sentiment_df['WeightedAverageSentimentScore'] = (sentiment_df['ticker_sentiment_score'] * sentiment_df['relevance_score']).sum() / sentiment_df['relevance_score'].sum()

    # drop the unnecessary columns
    sentiment_df.drop(columns=['relevance_score', 'ticker_sentiment_score', 'ticker_sentiment_label'], inplace=True)

    # group by ticker and calculate the mean for each group
    sentiment_df = sentiment_df.groupby('ticker').mean().reset_index()

    # round SentimentScore to nearest whole number and convert to integer
    sentiment_df['SentimentScore'] = round(sentiment_df['SentimentScore']).astype(int)

    # Create buy/sell signal
    sentiment_df['Signal'] = sentiment_df['SentimentScore'].apply(lambda x: 'Buy' if x >= 3 else ('Sell' if x <= 1 else 'Hold'))

    return sentiment_df

In [6]:
sentiment_dfs = []
for ticker in tickers:
    try:
        sentiment_df = get_sentiment_data(ticker, API_TOKEN)
        sentiment_dfs.append(sentiment_df)
    except Exception as e:
        print(f"Error getting sentiment data for {ticker}: {e}")

Error getting sentiment data for SONY: 'ticker_sentiment_label'


In [7]:
sentiment_dfs

[  ticker  SentimentScore  WeightedAverageSentimentScore Signal
 0   AAPL               3                       0.242145    Buy,
   ticker  SentimentScore  WeightedAverageSentimentScore Signal
 0   MSFT               2                        0.20333   Hold,
   ticker  SentimentScore  WeightedAverageSentimentScore Signal
 0   GOOG               2                       0.102743   Hold,
   ticker  SentimentScore  WeightedAverageSentimentScore Signal
 0   AMZN               3                       0.188497    Buy,
   ticker  SentimentScore  WeightedAverageSentimentScore Signal
 0   NVDA               3                       0.304965    Buy,
   ticker  SentimentScore  WeightedAverageSentimentScore Signal
 0   TSLA               2                        0.16408   Hold,
   ticker  SentimentScore  WeightedAverageSentimentScore Signal
 0   META               2                       0.043362   Hold,
   ticker  SentimentScore  WeightedAverageSentimentScore Signal
 0    TSM               2        